list 내장 타입에는 리스트의 원소를 정렬할 수 있는 sort 메서드가 있음

In [1]:
# Example 1
numbers = [93, 86, 11, 68, 70]
numbers.sort()
print(numbers)

[11, 68, 70, 86, 93]


sort는 객체를 어떻게 처리할까?

In [2]:
# Example 2
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight

    def __repr__(self):
        return f'Tool({self.name!r}, {self.weight})'

tools = [
    Tool('level', 3.5),
    Tool('hammer', 1.25),
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25),
]

sort 메서드가 호출하는 객체 비교 특별 메서드가 정의돼 있지 않으므로 이런 타입의 객체를 정렬할 수 없음 

In [3]:
tools.sort()

TypeError: '<' not supported between instances of 'Tool' and 'Tool'

정렬에 사용하고 싶은 attribute가 객체에 들어 있는 경우가 많다. 
이런 상황을 지원하기 위해 sort에는 key라는 파라미터가 있음. key는 함수여야 한다. key 함수에는 정렬 중인 리스트의 원소가 전달. key 함수가 반환하는 값은 원소 대신 정렬 기준으로 사용할 값이어야 함 

Tool 객체를 이름에 따라 정렬 

In [4]:
# Example 4
print('Unsorted:', repr(tools))
tools.sort(key=lambda x: x.name)
print('\nSorted:  ', tools)

Unsorted: [Tool('level', 3.5), Tool('hammer', 1.25), Tool('screwdriver', 0.5), Tool('chisel', 0.25)]

Sorted:   [Tool('chisel', 0.25), Tool('hammer', 1.25), Tool('level', 3.5), Tool('screwdriver', 0.5)]


In [8]:
# Example 5
tools.sort(key=lambda x: x.weight)
print('By weight:', tools)

By weight: [Tool('chisel', 0.25), Tool('screwdriver', 0.5), Tool('hammer', 1.25), Tool('level', 3.5)]


문자열 같은 기본 타입의 경우에는 정렬하기 전에 key 함수를 사용해 원소 값을 변형할 수 있음  

다음 예제는 lower 메서드를 사용해 리스트에 들어 있는 장소 이름을 소문자로 변환함으로써 첫 글자가 대문자든 소문자든 굽누하지 않고 알파벳순으로 비교 

In [10]:
# Example 6
places = ['home', 'work', 'New York', 'Paris']
places.sort()
print('Case sensitive:  ', places)
places.sort(key=lambda x: x.lower())
print('Case insensitive:', places)

Case sensitive:   ['New York', 'Paris', 'home', 'work']
Case insensitive: ['home', 'New York', 'Paris', 'work']


여러 기준을 사용해 정렬해야 되는 경우  -> __튜플을 사용하자__  

튜플은 비교 가능하며 자연스러운 순서가 정해져 있음  

튜플의 각 위치를 이터레이션하면서 각 인덱스에 해당하는 원소를 한 번에 하나씩 비교

In [11]:
# Example 7
power_tools = [
    Tool('drill', 4),
    Tool('circular saw', 5),
    Tool('jackhammer', 40),
    Tool('sander', 4),
]

In [12]:
# Example 8
saw = (5, 'circular saw')
jackhammer = (40, 'jackhammer')
assert not (jackhammer < saw)  # Matches expectations


두 튜플의 첫 번째 위치에 있는 값이 서로 같으면 두 번째 위치에 있는 값을 서로 비교... 세 번째 비교... 

In [14]:
# Example 9
drill = (4, 'drill')
sander = (4, 'sander')
assert drill[0] == sander[0]  # Same weight
assert drill[1] < sander[1]   # Alphabetically less
assert drill < sander         # Thus, drill comes first

weight 정렬 후 name으로 정렬하기

In [16]:
# Example 10
power_tools.sort(key=lambda x: (x.weight, x.name))
print(power_tools)


[Tool('drill', 4), Tool('sander', 4), Tool('circular saw', 5), Tool('jackhammer', 40)]


튜플을 반환하는 key 함수의 한 가지 제약 사항은 모든 비교 기준의 정렬 순서가 같다 (모두 오름차순 or 내림차순)

In [17]:
# Example 11
power_tools.sort(key=lambda x: (x.weight, x.name),
                 reverse=True)  # Makes all criteria descending
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('sander', 4), Tool('drill', 4)]


숫자 값의 경우 반전(-) 연산자를 사용해 정렬 방향을 혼합할 수 있음 

In [18]:
# Example 12
power_tools.sort(key=lambda x: (-x.weight, x.name))
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


하지만 모든 타입에 부호 반전을 사용할 수 없음 

In [20]:
import logging
# Example 13
try:
    power_tools.sort(key=lambda x: (x.weight, -x.name),
                     reverse=True)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-20-0703a4f00226>", line 4, in <module>
    power_tools.sort(key=lambda x: (x.weight, -x.name),
  File "<ipython-input-20-0703a4f00226>", line 4, in <lambda>
    power_tools.sort(key=lambda x: (x.weight, -x.name),
TypeError: bad operand type for unary -: 'str'


이런 상황을 위해서 안정적인(stable) 정렬 알고리즘 제공  

리스트 타입의 sort 메서드는 key 함수가 반환하는 값이 서로 같은 경우 리스트에 들어 있던 원래 순서를 그대로 유지  
이는 같은 리스트에 대해 서로 다른 기준으로 sort를 여러 번 호출해도 된다는 뜻  

weight 기준 내림차순, name 기준 오름차순으로 정렬하는 예시 

In [21]:
power_tools.sort(key=lambda x: x.name)   # Name ascending

power_tools.sort(key=lambda x: x.weight, # Weight descending
                 reverse=True)

In [22]:
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


주의할 점은 최종적으로 리스트에서 얻어내고 싶은 정렬 기준 우선순위 역순으로 정렬을 수행해야 한다